In [4]:
from swarm import Swarm, Agent

client = Swarm()

def transfer_to_agent_b():
    return agent_b


agent_a = Agent(
    name="Agent A",
    instructions="You are a helpful agent.",
    functions=[transfer_to_agent_b],
)

agent_b = Agent(
    name="Agent B",
    instructions="Only speak in Chinese.",
)

response = client.run(
    agent=agent_a,
    messages=[{"role": "user", "content": "I want to talk to agent B."}],
)

print(response.messages[-1]["content"])

我已将您转接给Agent B。请问有什么我可以帮助您的吗？


In [5]:
client.memory.model_dump()

{'memory': [{'messages': [{'content': None,
     'role': 'assistant',
     'function_call': None,
     'tool_calls': [{'id': 'call_WiCpk3aAlH9RTKlAOaO91hbk',
       'function': {'arguments': '{}', 'name': 'transfer_to_agent_b'},
       'type': 'function'}],
     'refusal': None,
     'sender': 'Agent A'},
    {'role': 'tool',
     'tool_call_id': 'call_WiCpk3aAlH9RTKlAOaO91hbk',
     'tool_name': 'transfer_to_agent_b',
     'content': '{"assistant": "Agent B"}'},
    {'content': '我已将您转接给Agent B。请问有什么我可以帮助您的吗？',
     'role': 'assistant',
     'function_call': None,
     'tool_calls': None,
     'refusal': None,
     'sender': 'Agent B'}],
   'agent': {'name': 'Agent B',
    'model': 'gpt-4o',
    'instructions': 'Only speak in Chinese.',
    'functions': [],
    'tool_choice': None,
    'parallel_tool_calls': True},
   'context_variables': {},
   'timestamp': '2024-10-14 11:45:27'}]}

In [6]:
from swarm.types import Result

sales_agent = Agent(name="Sales Agent")

def talk_to_sales():
   print("Hello, World!")
   return Result(
       value="Done",
       agent=sales_agent,
       context_variables={"department": "sales"}
   )

agent = Agent(functions=[talk_to_sales])

response = client.run(
   agent=agent,
   messages=[{"role": "user", "content": "Transfer me to sales"}],
   context_variables={"user_name": "John"}
)
print(response.agent.name)
print(response.context_variables)

Hello, World!
Sales Agent
{'user_name': 'John', 'department': 'sales'}


In [1]:
import json
from swarm import Swarm
from swarm.repl import process_and_print_streaming_response, pretty_print_messages
from examples.triage_agent.agents import triage_agent

In [4]:
client = Swarm()

messages = []
agent = triage_agent
stream = False
debug = False

while True:
    user_input = input("\033[90mUser\033[0m: ")
    messages.append({"role": "user", "content": user_input})

    response = client.run(
        agent=agent,
        messages=messages,
        context_variables={},
        stream=stream,
        debug=debug,
    )
    print('User: ', user_input)
    if stream:
        response = process_and_print_streaming_response(response)
    else:
        pretty_print_messages(response.messages)

    messages.extend(response.messages)
    agent = response.agent

User:  Hello!
Triage Agent: Hi there! How can I assist you today?
User:  i want to buy some beehives
Triage Agent: transfer_to_sales()
Sales Agent: Buzz-tastic choice! Beehives are an incredible investment for boosting pollination, producing honey, and supporting our buzzing friends. Let's get you started on making your beekeeping dreams a reality! How many beehives are you looking to purchase, and is there a specific type you're interested in? 🍯🐝
User:  give me 100 first, and looking into Warré hives
Sales Agent: Wow, 100 Warré hives? That's phenomenal! Warré hives are a fantastic choice due to their natural design, which closely mimics how bees build their hives in the wild. They're compact, sustainable, and require less interference, allowing your bees to thrive naturally.

With 100 hives, you’ll have a buzzing empire of productivity! 🐝🌼 Just imagine all the honey and the ecological benefit you’ll be contributing to the local environment. Let's make your beekeeping adventure as swee

KeyboardInterrupt: Interrupted by user

In [15]:
memory = client.memory.model_dump()['memory']
print("number of turns:", len(memory))

number of turns: 17


In [16]:
memory[0]

{'messages': [{'role': 'user', 'content': 'Hello!'},
  {'content': 'Hi there! How can I assist you today?',
   'role': 'assistant',
   'function_call': None,
   'tool_calls': None,
   'refusal': None,
   'sender': 'Triage Agent'}],
 'agent': {'name': 'Triage Agent',
  'model': 'gpt-4o',
  'instructions': "Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.",
  'functions': [<function examples.triage_agent.agents.transfer_to_sales()>,
   <function examples.triage_agent.agents.transfer_to_refunds()>],
  'tool_choice': None,
  'parallel_tool_calls': True},
 'context_variables': {},
 'timestamp': '2024-10-17 13:25:20'}

In [17]:
memory[-1]

{'messages': [{'role': 'user', 'content': 'bring me back '},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_Hi7K1Pxpbi1Z3saNF8gaev5U',
     'function': {'arguments': '{}', 'name': 'transfer_back_to_triage'},
     'type': 'function'}],
   'refusal': None,
   'sender': 'Refunds Agent'},
  {'role': 'tool',
   'tool_call_id': 'call_Hi7K1Pxpbi1Z3saNF8gaev5U',
   'tool_name': 'transfer_back_to_triage',
   'content': '{"assistant": "Triage Agent"}'},
  {'content': "You're back with me now! How can I assist you further?",
   'role': 'assistant',
   'function_call': None,
   'tool_calls': None,
   'refusal': None,
   'sender': 'Triage Agent'}],
 'agent': {'name': 'Triage Agent',
  'model': 'gpt-4o',
  'instructions': "Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.",
  'functions': [<function examples.triage_agent.agents.transfer_to_sales()>,
   <function examples.triage_agent.a